# Анализ NPS телекоммуникационной компании

<div class="alert alert-success">
<font size="5"><b>Подготовка данных к исследованию</b></font>

Данные находятся в SQLite — СУБД, в которой база данных представлена файлом. Для подключения к такой базе достаточно иметь доступ к файлу с расширением .db.
Чтобы подключиться к базе данных и сохранить данные в датафрейм в pandas, понадобятся библиотеки urllib & sqlalchemy.
    
Нам нужно собрать в одну витрину данные из разных таблиц. Эту витрину мы будем использовать для дашборда. В БД имеются таблицы user, location, age_segment, traffic_segment, lifetime_segment. Все таблицы связаны друг с другом с помощью id разных признаков. 
    
</div>

In [12]:
# Setting up the Environment

import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import urllib.request

In [13]:
# URL of the SQLite database
url = 'https://code.s3.yandex.net/datasets/telecomm_csi.db'
local_filename = 'telecomm_csi.db'

# Download the SQLite file
urllib.request.urlretrieve(url, local_filename)
engine = create_engine(f'sqlite:///{local_filename}', echo=False)

In [14]:
# Retrieve data from SQLite db
query = """
SELECT u.user_id, 
       u.lt_day, 
       u.is_new,
       u.age,
       u.gender_segment,
       u.os_name,
       u.cpe_type_name,
       u.country,
       u.city,
       u.age_segment,
       u.traffic_segment,
       substr(l.title, 4, 10) lifetime_segment,
       u.nps_score,
       u.nps_group
FROM (
    SELECT u.*, 
           substr(t.title, 4, 10) traffic_segment
    FROM (
        with u as
        (
        SELECT u.*, 
               CASE WHEN u.lt_day <= 365 THEN 'newcomer' ELSE 'old_user' END is_new,
               l.country,
               l.city,
               CASE WHEN u.nps_score <=6 THEN 'Detractor'
                    WHEN u.nps_score >=9 THEN 'Promoter'
                    ELSE 'Neutral'
                    END 'nps_group'
        FROM user u
        LEFT JOIN location l ON u.location_id = l.location_id
        )
        SELECT u.*,
               substr(a.title, 4, 10) age_segment           
        FROM u
        LEFT OUTER JOIN age_segment a ON u.age_gr_id = a.age_gr_id) AS u
    LEFT JOIN traffic_segment t ON u.tr_gr_id = t.tr_gr_id) AS u
LEFT JOIN lifetime_segment l ON u.lt_gr_id = l.lt_gr_id
;
"""

In [15]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old_user,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Promoter
1,A001WF,2344,old_user,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Promoter
2,A003Q7,467,old_user,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Promoter


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

<div class="alert alert-success">
<font size="5"><b>Обзор данных</b></font>

** С помощью запроса SQL выгрузили все необходимые для анализа поля из БД. Получившийся датафрейм выгружаем в виде .csv файла, и на основе этих данных будем строить дэшборды в Tableau.


| поле | описание |
| ----------- | ----------- |
| user_id |	Идентификатор клиента |
| lt_day |	Количество дней «жизни» клиента |
| is_new |	Поле хранит информацию о том, является ли клиент новым |
| age |	Возраст |
| gender_segment |	Пол (для удобства работы с полем преобразуйте значения в текстовый вид) |
| os_name |	Тип операционной системы |
| cpe_type_name |	Тип устройства |
| country |	Страна проживания |
| city |	Город проживания |
| age_segment |	Возрастной сегмент |
| traffic_segment |	Сегмент по объёму потребляемого трафика |
| lifetime_segment |	Сегмент по количеству дней «жизни» |
| nps_score |	Оценка клиента в NPS-опросе |
| nps_group |	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе |
    

</div>